In [1]:
import numpy as np
import altair as alt
import pandas as pd
import os

alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

## Load Data

In [2]:
sales_df = pd.read_csv("../data/raw/sales_and_customer_insights.csv", parse_dates=True)

## Basic Data Overview

In [3]:
sales_df.head()

,Customer_ID,Product_ID,Transaction_ID,Purchase_Frequency,Average_Order_Value,Most_Frequent_Category,Time_Between_Purchases,Region,Churn_Probability,Lifetime_Value,Launch_Date,Peak_Sales_Date,Season,Preferred_Purchase_Times,Retention_Strategy
0,CUST_9HOS83,PROD_IK97D1,TRANS_II1DZG,17,172.57,Electronics,45,South America,0.98,952.81,2020-03-14,2023-09-11,Winter,Afternoon,Loyalty Program
1,CUST_AJU17N,PROD_UNN7KP,TRANS_9HJF7I,10,64.89,Clothing,6,South America,0.66,5427.51,2022-10-15,2023-01-02,Spring,Afternoon,Discount
2,CUST_11XNYF,PROD_0XEW2W,TRANS_OT96OM,3,120.38,Sports,23,Asia,0.60,3994.80,2021-11-30,2023-04-06,Winter,Evening,Loyalty Program
3,CUST_IGH8G3,PROD_3IIAJN,TRANS_45V00G,12,70.34,Clothing,5,North America,0.78,4880.01,2022-03-20,2023-03-23,Spring,Evening,Discount
4,CUST_OK6PUM,PROD_VMIWD2,TRANS_ZAK760,18,42.39,Electronics,10,North America,0.52,8557.70,2022-11-09,2023-10-28,Spring,Morning,Discount


In [19]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Customer_ID               10000 non-null  object 
 1   Product_ID                10000 non-null  object 
 2   Transaction_ID            10000 non-null  object 
 3   Purchase_Frequency        10000 non-null  int64  
 4   Average_Order_Value       10000 non-null  float64
 5   Most_Frequent_Category    10000 non-null  object 
 6   Time_Between_Purchases    10000 non-null  int64  
 7   Region                    10000 non-null  object 
 8   Churn_Probability         10000 non-null  float64
 9   Lifetime_Value            10000 non-null  float64
 10  Launch_Date               10000 non-null  object 
 11  Peak_Sales_Date           10000 non-null  object 
 12  Season                    10000 non-null  object 
 13  Preferred_Purchase_Times  10000 non-null  object 
 14  Retenti

In [4]:
sales_df.describe()

,Purchase_Frequency,Average_Order_Value,Time_Between_Purchases,Churn_Probability,Lifetime_Value
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,9.955700,110.006022,46.885300,0.501552,5031.930567
std,5.526209,52.096870,24.711746,0.288289,2865.420299
min,1.000000,20.010000,5.000000,0.000000,100.160000
25%,5.000000,65.262500,25.000000,0.250000,2518.775000
50%,10.000000,109.930000,47.000000,0.500000,5012.180000
75%,15.000000,155.480000,68.000000,0.750000,7526.232500
max,19.000000,199.960000,89.000000,1.000000,9999.760000


## Preprocessing

In [5]:
sales_df["high_churn_risk"] = sales_df["Churn_Probability"] >= 0.75

In [6]:
sales_100_sample_df = sales_df.sample(100, random_state = 448)
sales_1000_sample_df = sales_df.sample(1000, random_state = 448)

In [7]:
sales_100_sample_df["high_churn_alpha"] = 0.3
sales_100_sample_df.loc[sales_100_sample_df["high_churn_risk"] == True, "high_churn_alpha"] = 0.8

In [8]:
sales_100_sample_df

,Customer_ID,Product_ID,Transaction_ID,Purchase_Frequency,Average_Order_Value,Most_Frequent_Category,Time_Between_Purchases,Region,Churn_Probability,Lifetime_Value,Launch_Date,Peak_Sales_Date,Season,Preferred_Purchase_Times,Retention_Strategy,high_churn_risk,high_churn_alpha
4538,CUST_LTJQFU,PROD_E4XF14,TRANS_AN8QBX,12,160.34,Home,48,South America,0.32,1368.98,2021-10-03,2023-01-04,Spring,Evening,Loyalty Program,False,0.3
3149,CUST_U42MFP,PROD_T4IN56,TRANS_KLHWZ7,19,86.28,Electronics,51,South America,0.94,7261.40,2021-12-29,2023-09-09,Fall,Morning,Loyalty Program,True,0.8
7139,CUST_ECZY3B,PROD_HZZV34,TRANS_UBDNY5,19,35.44,Clothing,60,Europe,0.23,3505.73,2021-05-08,2023-10-16,Summer,Evening,Email Campaign,False,0.3
6435,CUST_FC992O,PROD_NR8W33,TRANS_BNIV6Q,6,32.72,Home,52,North America,0.02,7276.74,2020-04-12,2023-01-31,Fall,Morning,Email Campaign,False,0.3
7634,CUST_KGSHM8,PROD_MX138D,TRANS_VWHHR2,4,174.74,Home,27,North America,0.89,7684.37,2020-04-18,2023-07-21,Spring,Morning,Email Campaign,True,0.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2384,CUST_C7UNBK,PROD_CDY9YA,TRANS_WTD47J,9,64.93,Sports,41,North America,0.65,3318.27,2021-11-12,2023-01-01,Summer,Afternoon,Email Campaign,False,0.3
7568,CUST_C21E6Q,PROD_KDPZNE,TRANS_AYGWE8,17,35.90,Home,41,Europe,0.79,9211.32,2020-08-11,2023-11-20,Summer,Morning,Loyalty Program,True,0.8
6506,CUST_WOYI3R,PROD_L8A1A9,TRANS_4YJ689,4,95.85,Clothing,44,North America,0.02,1938.29,2020-01-16,2023-03-11,Summer,Afternoon,Loyalty Program,False,0.3
9435,CUST_74VIOR,PROD_CSER1S,TRANS_H4OPMB,16,76.90,Electronics,40,Europe,0.05,4359.55,2022-06-04,2023-07-31,Summer,Evening,Loyalty Program,False,0.3


In [9]:
# High-Value customers (Assuming they are those who have Lifetime_Value > $7000)
high_value_df = sales_df[sales_df['Lifetime_Value'] > 7000]

# Analyzing distribution of strategies for those at high churn risk within this group
at_risk_high_value = high_value_df[high_value_df['high_churn_risk'] == True]

# Analyzing the distribution of retention strategies for the high-value, high-risk segment.
strategy_counts = at_risk_high_value['Retention_Strategy'].value_counts()

print("Current Retention Strategies for High-Value At-Risk Segment:")
print(strategy_counts)

Current Retention Strategies for High-Value At-Risk Segment:
Retention_Strategy
Email Campaign     262
Loyalty Program    254
Discount           253
Name: count, dtype: int64


## Charts For Dashboard Markup

In [12]:
story_2_example = alt.Chart(sales_1000_sample_df).mark_circle(size = 50).encode(
    alt.X('Lifetime_Value', title='Lifetime Value'),
    alt.Y('Time_Between_Purchases', title='Average Days Between Purchases'),
    color = 'Retention_Strategy',
    opacity = 'Churn_Probability'
).properties(
    title = alt.Title('Retension Strategies (1000 point sample)',fontSize=20),
    height = 400,
    width = 400,
).configure_axis(
    labelFontSize=16,
    titleFontSize=20
)
story_2_example

alt.Chart(...)

In [13]:
story_3_example = alt.Chart(sales_df).transform_aggregate(
    mean_freq = 'mean(Purchase_Frequency)',
    groupby=["Season","Most_Frequent_Category"]
).mark_rect(color = "viridis").encode(
    alt.X('Season', title='Season'),
    alt.Y('Most_Frequent_Category', title='Product Type'),
    color = "mean_freq:Q"
).properties(
    title = 'Mean Purchasing Frequency by Season and Category',
    height = 400,
    width = 400,
)
story_3_example

alt.Chart(...)